In [1]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import torch.optim as optim
import random
import time
import json
import copy

from load_data import load_dataset

SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### SingleCNN Model Structure

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class SingleCNN(nn.Module):
    def __init__(self, mode, vocab_size, embedding_dim, n_filters, filter_size, output_dim, dropout, pad_idx):
        
        super().__init__()
        self.mode = mode
        self.embed_size = embedding_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        if self.mode == "non-static":
            self.embedding.weight.requires_grad = True
        else:
            self.embedding.weight.requires_grad = False
        self.conv = nn.Conv2d(in_channels = 1, out_channels = n_filters, kernel_size = (filter_size, embedding_dim), padding = (filter_size - 1, 0))
        self.fc = nn.Linear(n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)

    def pe(self,M):
        embed_size = self.embed_size
        n_even = (embed_size+1)//2
        n_odd = embed_size-n_even
        p = torch.tensor([m for m in range(M)])
        isin = torch.tensor([i for i in range(n_even)])
        icos = torch.tensor([i for i in range(n_odd)])
        Psin, Isin = torch.meshgrid(p, isin)#, indexing='ij'
        Pcos, Icos = torch.meshgrid(p, icos)
        Sin = torch.sin(Psin/(10000**(2*Isin/embed_size)))
        Cos = torch.cos(Pcos/(10000**(2*Icos/embed_size)))
        PE = torch.zeros(M,embed_size)
        PE[ : , [i*2 for i in range(n_even)]] = Sin
        PE[ : , [i*2 + 1 for i in range(n_odd)]] = Cos
        return PE

        
    def forward(self, text):
        _, M=text.size()
        embedded = self.embedding(text)
        if self.mode == "spe":
            embedded += self.pe(M).unsqueeze(0).cuda()
        #text = [batch size, sent len]
        # embedded = self.embedding(text)
        #embedded = [batch size, sent len, emb dim]
        # embedded += self.pe(M).unsqueeze(0).cuda()
        embedded = embedded.unsqueeze(1)
        #embedded = [batch size, 1, sent len, emb dim]
        #conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        conved = F.relu(self.conv(embedded).squeeze(3))
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        maxpooled = F.max_pool1d(conved, conved.shape[2]).squeeze(2)
        avgpooled = F.avg_pool1d(conved, conved.shape[2]).squeeze(2)
        pooled = maxpooled + avgpooled
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(pooled)
        #cat = [batch size, 2 * n_filters * len(filter_sizes)]
        return self.fc(cat)

In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label.long())
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label.long())
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def multi_models(model, train_iterator, valid_iterator,  test_iterator, num = 5, N_EPOCHS = 50, early_stopping = 5):
    init_model = copy.deepcopy(model)
    res = {}
    res["test_loss"] = []
    res["test_acc"] = []
    ### begin training process
    for i in range(num):
        print('Begin training model %s'%i)
        res["model_%s"%i] = {}
        res["model_%s"%i]["time"] = []
        res["model_%s"%i]["train_loss"] = []
        res["model_%s"%i]["val_loss"] = []
        res["model_%s"%i]["train_acc"] = []
        res["model_%s"%i]["val_acc"] = []
        best_valid_loss = float('inf')
        num_steps = 0
        ### reset model for each round
        # model.load_state_dict(init_param)
        model = copy.deepcopy(init_model)
        model = model.to(device)
        optimizer = optim.Adam(model.parameters())
        criterion = nn.CrossEntropyLoss()
        criterion = criterion.to(device)
        
        for epoch in range(N_EPOCHS):
            print('Begin epoch %s'%epoch)
            start_time = time.time()
            train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
            valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
            end_time = time.time()
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                torch.save(model.state_dict(), 'scnn-model.pt')
                num_steps = 0
            else:
                num_steps += 1
            print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
            print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
            res["model_%s"%i]["time"].append(end_time - start_time)
            res["model_%s"%i]["train_loss"].append(train_loss)
            res["model_%s"%i]["val_loss"].append(valid_loss)
            res["model_%s"%i]["train_acc"].append(train_acc)
            res["model_%s"%i]["val_acc"].append(valid_acc)
            if num_steps >= early_stopping:
                break
        
        model.load_state_dict(torch.load('scnn-model.pt'))
        test_loss, test_acc = evaluate(model, test_iterator, criterion)
        print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
        res["test_loss"].append(test_loss)
        res["test_acc"].append(test_acc)
    return res

### SingleCNN wiht filter size 3

In [4]:
data_name = "TREC"
train_iterator, valid_iterator, test_iterator, TEXT, LABEL = load_dataset(data_name, device)

### Statis Model without SPE
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 128
FILTER_SIZE = 3
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

MODE = "static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_static_fs3_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "non-static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_nonstatic_fs3_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "spe"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_spe_fs3_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))

Begin training model 0
Begin epoch 0
	Train Loss: 1.204 | Train Acc: 55.56%
	 Val. Loss: 0.723 |  Val. Acc: 79.39%
Begin epoch 1


/hpc/group/jilab/changxin/miniconda3/envs/py37/lib/python3.7/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


	Train Loss: 0.680 | Train Acc: 76.97%
	 Val. Loss: 0.504 |  Val. Acc: 85.44%
Begin epoch 2
	Train Loss: 0.514 | Train Acc: 82.44%
	 Val. Loss: 0.431 |  Val. Acc: 84.59%
Begin epoch 3
	Train Loss: 0.409 | Train Acc: 86.66%
	 Val. Loss: 0.398 |  Val. Acc: 85.62%
Begin epoch 4
	Train Loss: 0.350 | Train Acc: 88.23%
	 Val. Loss: 0.375 |  Val. Acc: 86.85%
Begin epoch 5
	Train Loss: 0.293 | Train Acc: 90.60%
	 Val. Loss: 0.351 |  Val. Acc: 88.40%
Begin epoch 6
	Train Loss: 0.250 | Train Acc: 91.67%
	 Val. Loss: 0.347 |  Val. Acc: 88.05%
Begin epoch 7
	Train Loss: 0.222 | Train Acc: 92.75%
	 Val. Loss: 0.360 |  Val. Acc: 88.06%
Begin epoch 8
	Train Loss: 0.196 | Train Acc: 94.44%
	 Val. Loss: 0.336 |  Val. Acc: 88.41%
Begin epoch 9
	Train Loss: 0.180 | Train Acc: 94.40%
	 Val. Loss: 0.336 |  Val. Acc: 88.06%
Begin epoch 10
	Train Loss: 0.158 | Train Acc: 94.88%
	 Val. Loss: 0.333 |  Val. Acc: 89.96%
Begin epoch 11
	Train Loss: 0.145 | Train Acc: 95.69%
	 Val. Loss: 0.335 |  Val. Acc: 88.77%


In [5]:
data_name = "SST"
train_iterator, valid_iterator, test_iterator, TEXT, LABEL = load_dataset(data_name, device)

### Statis Model without SPE
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 128
FILTER_SIZE = 3
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

MODE = "static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_static_fs3_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "non-static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_nonstatic_fs3_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "spe"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_spe_fs3_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))

Begin training model 0
Begin epoch 0
	Train Loss: 0.303 | Train Acc: 87.48%
	 Val. Loss: 0.403 |  Val. Acc: 82.19%
Begin epoch 1
	Train Loss: 0.245 | Train Acc: 90.38%
	 Val. Loss: 0.411 |  Val. Acc: 81.07%
Begin epoch 2
	Train Loss: 0.218 | Train Acc: 91.63%
	 Val. Loss: 0.458 |  Val. Acc: 80.54%
Begin epoch 3
	Train Loss: 0.201 | Train Acc: 92.38%
	 Val. Loss: 0.442 |  Val. Acc: 80.96%
Begin epoch 4
	Train Loss: 0.188 | Train Acc: 93.02%
	 Val. Loss: 0.468 |  Val. Acc: 80.89%
Begin epoch 5
	Train Loss: 0.179 | Train Acc: 93.42%
	 Val. Loss: 0.429 |  Val. Acc: 82.81%
Test Loss: 0.386 | Test Acc: 82.66%
Begin training model 1
Begin epoch 0
	Train Loss: 0.304 | Train Acc: 87.53%
	 Val. Loss: 0.443 |  Val. Acc: 80.56%
Begin epoch 1
	Train Loss: 0.244 | Train Acc: 90.47%
	 Val. Loss: 0.407 |  Val. Acc: 82.79%
Begin epoch 2
	Train Loss: 0.219 | Train Acc: 91.58%
	 Val. Loss: 0.430 |  Val. Acc: 82.41%
Begin epoch 3
	Train Loss: 0.199 | Train Acc: 92.50%
	 Val. Loss: 0.433 |  Val. Acc: 82.23

In [6]:
data_name = "IMDB"
train_iterator, valid_iterator, test_iterator, TEXT, LABEL = load_dataset(data_name, device)

### Statis Model without SPE
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 128
FILTER_SIZE = 3
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

MODE = "static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_static_fs3_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "non-static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_nonstatic_fs3_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "spe"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_spe_fs3_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))

Begin training model 0
Begin epoch 0
	Train Loss: 0.495 | Train Acc: 75.10%
	 Val. Loss: 0.347 |  Val. Acc: 85.31%
Begin epoch 1
	Train Loss: 0.350 | Train Acc: 85.00%
	 Val. Loss: 0.305 |  Val. Acc: 86.91%
Begin epoch 2
	Train Loss: 0.311 | Train Acc: 86.67%
	 Val. Loss: 0.299 |  Val. Acc: 87.34%
Begin epoch 3
	Train Loss: 0.284 | Train Acc: 88.27%
	 Val. Loss: 0.281 |  Val. Acc: 87.62%
Begin epoch 4
	Train Loss: 0.256 | Train Acc: 89.48%
	 Val. Loss: 0.270 |  Val. Acc: 88.05%
Begin epoch 5
	Train Loss: 0.242 | Train Acc: 90.25%
	 Val. Loss: 0.270 |  Val. Acc: 88.36%
Begin epoch 6
	Train Loss: 0.224 | Train Acc: 91.12%
	 Val. Loss: 0.275 |  Val. Acc: 87.97%
Begin epoch 7
	Train Loss: 0.204 | Train Acc: 91.90%
	 Val. Loss: 0.270 |  Val. Acc: 88.36%
Begin epoch 8
	Train Loss: 0.189 | Train Acc: 92.68%
	 Val. Loss: 0.285 |  Val. Acc: 88.16%
Begin epoch 9
	Train Loss: 0.183 | Train Acc: 92.81%
	 Val. Loss: 0.285 |  Val. Acc: 87.70%
Begin epoch 10
	Train Loss: 0.172 | Train Acc: 93.25%
	 V

### SingleCNN with filter size 6

In [7]:
data_name = "TREC"
train_iterator, valid_iterator, test_iterator, TEXT, LABEL = load_dataset(data_name, device)

FILTER_SIZE = 6
### Statis Model without SPE
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 128
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

MODE = "static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_static_fs6_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "non-static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_nonstatic_fs6_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "spe"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_spe_fs6_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))

Begin training model 0
Begin epoch 0
	Train Loss: 1.123 | Train Acc: 58.53%
	 Val. Loss: 0.643 |  Val. Acc: 80.09%
Begin epoch 1
	Train Loss: 0.599 | Train Acc: 79.76%
	 Val. Loss: 0.463 |  Val. Acc: 86.14%
Begin epoch 2
	Train Loss: 0.416 | Train Acc: 86.08%
	 Val. Loss: 0.397 |  Val. Acc: 86.16%
Begin epoch 3
	Train Loss: 0.327 | Train Acc: 89.41%
	 Val. Loss: 0.366 |  Val. Acc: 87.03%
Begin epoch 4
	Train Loss: 0.249 | Train Acc: 92.18%
	 Val. Loss: 0.361 |  Val. Acc: 87.38%
Begin epoch 5
	Train Loss: 0.201 | Train Acc: 93.62%
	 Val. Loss: 0.326 |  Val. Acc: 89.44%
Begin epoch 6
	Train Loss: 0.163 | Train Acc: 95.08%
	 Val. Loss: 0.323 |  Val. Acc: 88.06%
Begin epoch 7
	Train Loss: 0.131 | Train Acc: 96.53%
	 Val. Loss: 0.324 |  Val. Acc: 88.93%
Begin epoch 8
	Train Loss: 0.109 | Train Acc: 97.42%
	 Val. Loss: 0.322 |  Val. Acc: 88.59%
Begin epoch 9
	Train Loss: 0.093 | Train Acc: 97.93%
	 Val. Loss: 0.323 |  Val. Acc: 89.10%
Begin epoch 10
	Train Loss: 0.081 | Train Acc: 97.85%
	 V

In [8]:
data_name = "SST"
train_iterator, valid_iterator, test_iterator, TEXT, LABEL = load_dataset(data_name, device)

FILTER_SIZE = 6
### Statis Model without SPE
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 128
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

MODE = "static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_static_fs6_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "non-static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_nonstatic_fs6_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "spe"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_spe_fs6_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))

Begin training model 0
Begin epoch 0
	Train Loss: 0.293 | Train Acc: 88.03%
	 Val. Loss: 0.443 |  Val. Acc: 81.88%
Begin epoch 1
	Train Loss: 0.225 | Train Acc: 91.25%
	 Val. Loss: 0.436 |  Val. Acc: 81.50%
Begin epoch 2
	Train Loss: 0.191 | Train Acc: 92.81%
	 Val. Loss: 0.509 |  Val. Acc: 80.45%
Begin epoch 3
	Train Loss: 0.173 | Train Acc: 93.59%
	 Val. Loss: 0.548 |  Val. Acc: 78.35%
Begin epoch 4
	Train Loss: 0.158 | Train Acc: 94.25%
	 Val. Loss: 0.460 |  Val. Acc: 83.26%
Begin epoch 5
	Train Loss: 0.145 | Train Acc: 94.71%
	 Val. Loss: 0.480 |  Val. Acc: 82.39%
Begin epoch 6
	Train Loss: 0.138 | Train Acc: 95.07%
	 Val. Loss: 0.451 |  Val. Acc: 83.64%
Test Loss: 0.377 | Test Acc: 82.50%
Begin training model 1
Begin epoch 0
	Train Loss: 0.294 | Train Acc: 87.99%
	 Val. Loss: 0.430 |  Val. Acc: 80.51%
Begin epoch 1
	Train Loss: 0.223 | Train Acc: 91.30%
	 Val. Loss: 0.457 |  Val. Acc: 82.03%
Begin epoch 2
	Train Loss: 0.192 | Train Acc: 92.70%
	 Val. Loss: 0.472 |  Val. Acc: 81.79

In [9]:
data_name = "IMDB"
train_iterator, valid_iterator, test_iterator, TEXT, LABEL = load_dataset(data_name, device)

FILTER_SIZE = 6
### Statis Model without SPE
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 128
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

MODE = "static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_static_fs6_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "non-static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_nonstatic_fs6_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "spe"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_spe_fs6_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))

Begin training model 0
Begin epoch 0
	Train Loss: 0.457 | Train Acc: 77.30%
	 Val. Loss: 0.317 |  Val. Acc: 86.48%
Begin epoch 1
	Train Loss: 0.320 | Train Acc: 86.47%
	 Val. Loss: 0.283 |  Val. Acc: 87.70%
Begin epoch 2
	Train Loss: 0.273 | Train Acc: 88.53%
	 Val. Loss: 0.277 |  Val. Acc: 88.09%
Begin epoch 3
	Train Loss: 0.238 | Train Acc: 90.45%
	 Val. Loss: 0.271 |  Val. Acc: 88.32%
Begin epoch 4
	Train Loss: 0.209 | Train Acc: 91.91%
	 Val. Loss: 0.272 |  Val. Acc: 88.98%
Begin epoch 5
	Train Loss: 0.181 | Train Acc: 92.61%
	 Val. Loss: 0.281 |  Val. Acc: 88.32%
Begin epoch 6
	Train Loss: 0.157 | Train Acc: 93.93%
	 Val. Loss: 0.276 |  Val. Acc: 88.40%
Begin epoch 7
	Train Loss: 0.143 | Train Acc: 94.74%
	 Val. Loss: 0.283 |  Val. Acc: 89.38%
Begin epoch 8
	Train Loss: 0.129 | Train Acc: 95.24%
	 Val. Loss: 0.303 |  Val. Acc: 88.79%
Test Loss: 0.269 | Test Acc: 88.73%
Begin training model 1
Begin epoch 0
	Train Loss: 0.461 | Train Acc: 77.45%
	 Val. Loss: 0.320 |  Val. Acc: 86.41

### SingleCNN with filter size 12

In [10]:
data_name = "TREC"
train_iterator, valid_iterator, test_iterator, TEXT, LABEL = load_dataset(data_name, device)

FILTER_SIZE = 12
### Statis Model without SPE
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 128
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

MODE = "static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_static_fs12_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "non-static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_nonstatic_fs12_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "spe"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_spe_fs12_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))

Begin training model 0
Begin epoch 0
	Train Loss: 1.060 | Train Acc: 61.10%
	 Val. Loss: 0.612 |  Val. Acc: 80.96%
Begin epoch 1
	Train Loss: 0.548 | Train Acc: 81.13%
	 Val. Loss: 0.453 |  Val. Acc: 85.83%
Begin epoch 2
	Train Loss: 0.373 | Train Acc: 87.76%
	 Val. Loss: 0.399 |  Val. Acc: 85.46%
Begin epoch 3
	Train Loss: 0.257 | Train Acc: 92.38%
	 Val. Loss: 0.365 |  Val. Acc: 86.35%
Begin epoch 4
	Train Loss: 0.188 | Train Acc: 94.75%
	 Val. Loss: 0.356 |  Val. Acc: 86.84%
Begin epoch 5
	Train Loss: 0.145 | Train Acc: 96.05%
	 Val. Loss: 0.342 |  Val. Acc: 87.73%
Begin epoch 6
	Train Loss: 0.114 | Train Acc: 97.06%
	 Val. Loss: 0.355 |  Val. Acc: 86.51%
Begin epoch 7
	Train Loss: 0.093 | Train Acc: 97.73%
	 Val. Loss: 0.367 |  Val. Acc: 85.48%
Begin epoch 8
	Train Loss: 0.069 | Train Acc: 98.48%
	 Val. Loss: 0.341 |  Val. Acc: 87.90%
Begin epoch 9
	Train Loss: 0.055 | Train Acc: 98.87%
	 Val. Loss: 0.362 |  Val. Acc: 87.54%
Begin epoch 10
	Train Loss: 0.049 | Train Acc: 98.99%
	 V

In [11]:
data_name = "SST"
train_iterator, valid_iterator, test_iterator, TEXT, LABEL = load_dataset(data_name, device)

FILTER_SIZE = 12
### Statis Model without SPE
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 128
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

MODE = "static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_static_fs12_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "non-static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_nonstatic_fs12_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "spe"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_spe_fs12_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))

Begin training model 0
Begin epoch 0
	Train Loss: 0.288 | Train Acc: 88.21%
	 Val. Loss: 0.428 |  Val. Acc: 80.11%
Begin epoch 1
	Train Loss: 0.211 | Train Acc: 91.90%
	 Val. Loss: 0.429 |  Val. Acc: 81.27%
Begin epoch 2
	Train Loss: 0.174 | Train Acc: 93.47%
	 Val. Loss: 0.505 |  Val. Acc: 79.13%
Begin epoch 3
	Train Loss: 0.151 | Train Acc: 94.56%
	 Val. Loss: 0.491 |  Val. Acc: 80.56%
Begin epoch 4
	Train Loss: 0.136 | Train Acc: 95.12%
	 Val. Loss: 0.502 |  Val. Acc: 82.10%
Begin epoch 5
	Train Loss: 0.124 | Train Acc: 95.60%
	 Val. Loss: 0.492 |  Val. Acc: 80.09%
Test Loss: 0.388 | Test Acc: 82.35%
Begin training model 1
Begin epoch 0
	Train Loss: 0.291 | Train Acc: 88.08%
	 Val. Loss: 0.444 |  Val. Acc: 80.60%
Begin epoch 1
	Train Loss: 0.211 | Train Acc: 91.96%
	 Val. Loss: 0.434 |  Val. Acc: 82.05%
Begin epoch 2
	Train Loss: 0.175 | Train Acc: 93.42%
	 Val. Loss: 0.463 |  Val. Acc: 80.67%
Begin epoch 3
	Train Loss: 0.151 | Train Acc: 94.52%
	 Val. Loss: 0.484 |  Val. Acc: 80.49

In [4]:
data_name = "IMDB"
train_iterator, valid_iterator, test_iterator, TEXT, LABEL = load_dataset(data_name, device)

FILTER_SIZE = 12
### Statis Model without SPE
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 128
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

MODE = "static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_static_fs12_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "non-static"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_nonstatic_fs12_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))
    
MODE = "spe"
model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
### pre-train embeddings
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("SingleCNN_spe_fs12_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))

Begin training model 0
Begin epoch 0


/hpc/group/jilab/changxin/miniconda3/envs/py37/lib/python3.7/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


	Train Loss: 0.472 | Train Acc: 76.64%
	 Val. Loss: 0.352 |  Val. Acc: 84.96%
Begin epoch 1
	Train Loss: 0.319 | Train Acc: 86.37%
	 Val. Loss: 0.289 |  Val. Acc: 88.16%
Begin epoch 2
	Train Loss: 0.267 | Train Acc: 89.06%
	 Val. Loss: 0.279 |  Val. Acc: 88.05%
Begin epoch 3
	Train Loss: 0.216 | Train Acc: 91.39%
	 Val. Loss: 0.283 |  Val. Acc: 87.93%
Begin epoch 4
	Train Loss: 0.181 | Train Acc: 93.08%
	 Val. Loss: 0.290 |  Val. Acc: 88.83%
Begin epoch 5
	Train Loss: 0.151 | Train Acc: 94.10%
	 Val. Loss: 0.303 |  Val. Acc: 88.59%
Begin epoch 6
	Train Loss: 0.122 | Train Acc: 95.32%
	 Val. Loss: 0.324 |  Val. Acc: 88.16%
Begin epoch 7
	Train Loss: 0.106 | Train Acc: 96.22%
	 Val. Loss: 0.330 |  Val. Acc: 88.24%
Test Loss: 0.284 | Test Acc: 88.11%
Begin training model 1
Begin epoch 0
	Train Loss: 0.469 | Train Acc: 77.25%
	 Val. Loss: 0.338 |  Val. Acc: 85.23%
Begin epoch 1
	Train Loss: 0.322 | Train Acc: 86.24%
	 Val. Loss: 0.291 |  Val. Acc: 88.16%
Begin epoch 2
	Train Loss: 0.261 | 

### with filter size as 2

In [6]:
# data_name = "TREC"
FILTER_SIZE = 2
for data_name in ["TREC", "SST", "IMDB"]:
    train_iterator, valid_iterator, test_iterator, TEXT, LABEL = load_dataset(data_name, device)
    ### Statis Model without SPE
    INPUT_DIM = len(TEXT.vocab)
    EMBEDDING_DIM = 300
    N_FILTERS = 128
    OUTPUT_DIM = len(LABEL.vocab)
    DROPOUT = 0.5
    PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

    MODE = "static"
    model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
    ### pre-train embeddings
    pretrained_embeddings = TEXT.vocab.vectors
    model.embedding.weight.data.copy_(pretrained_embeddings)
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

    res = multi_models(model, train_iterator, valid_iterator, test_iterator)
    res["num_param"] = count_parameters(model)
    with open("SingleCNN_static_fs%s_%s.json"%(FILTER_SIZE, data_name), "w") as outfile:
        outfile.write(json.dumps(res, indent=4))

    MODE = "non-static"
    model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
    ### pre-train embeddings
    pretrained_embeddings = TEXT.vocab.vectors
    model.embedding.weight.data.copy_(pretrained_embeddings)
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

    res = multi_models(model, train_iterator, valid_iterator, test_iterator)
    res["num_param"] = count_parameters(model)
    with open("SingleCNN_nonstatic_fs%s_%s.json"%(FILTER_SIZE, data_name), "w") as outfile:
        outfile.write(json.dumps(res, indent=4))

    MODE = "spe"
    model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
    ### pre-train embeddings
    pretrained_embeddings = TEXT.vocab.vectors
    model.embedding.weight.data.copy_(pretrained_embeddings)
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

    res = multi_models(model, train_iterator, valid_iterator, test_iterator)
    res["num_param"] = count_parameters(model)
    with open("SingleCNN_spe_fs%s_%s.json"%(FILTER_SIZE, data_name), "w") as outfile:
        outfile.write(json.dumps(res, indent=4))

Begin training model 0
Begin epoch 0
	Train Loss: 1.259 | Train Acc: 52.77%
	 Val. Loss: 0.778 |  Val. Acc: 77.84%
Begin epoch 1
	Train Loss: 0.749 | Train Acc: 74.56%
	 Val. Loss: 0.543 |  Val. Acc: 82.33%
Begin epoch 2
	Train Loss: 0.570 | Train Acc: 80.85%
	 Val. Loss: 0.451 |  Val. Acc: 84.43%
Begin epoch 3
	Train Loss: 0.467 | Train Acc: 84.68%
	 Val. Loss: 0.411 |  Val. Acc: 85.81%
Begin epoch 4
	Train Loss: 0.397 | Train Acc: 86.54%
	 Val. Loss: 0.382 |  Val. Acc: 85.98%
Begin epoch 5
	Train Loss: 0.350 | Train Acc: 88.37%
	 Val. Loss: 0.358 |  Val. Acc: 86.49%
Begin epoch 6
	Train Loss: 0.309 | Train Acc: 90.19%
	 Val. Loss: 0.346 |  Val. Acc: 87.88%
Begin epoch 7
	Train Loss: 0.281 | Train Acc: 90.84%
	 Val. Loss: 0.357 |  Val. Acc: 87.37%
Begin epoch 8
	Train Loss: 0.259 | Train Acc: 91.34%
	 Val. Loss: 0.335 |  Val. Acc: 88.75%
Begin epoch 9
	Train Loss: 0.243 | Train Acc: 92.24%
	 Val. Loss: 0.329 |  Val. Acc: 88.06%
Begin epoch 10
	Train Loss: 0.208 | Train Acc: 93.53%
	 V

In [7]:
# data_name = "TREC"
FILTER_SIZE = 4
for data_name in ["TREC", "SST", "IMDB"]:
    train_iterator, valid_iterator, test_iterator, TEXT, LABEL = load_dataset(data_name, device)
    ### Statis Model without SPE
    INPUT_DIM = len(TEXT.vocab)
    EMBEDDING_DIM = 300
    N_FILTERS = 128
    OUTPUT_DIM = len(LABEL.vocab)
    DROPOUT = 0.5
    PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

    MODE = "static"
    model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
    ### pre-train embeddings
    pretrained_embeddings = TEXT.vocab.vectors
    model.embedding.weight.data.copy_(pretrained_embeddings)
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

    res = multi_models(model, train_iterator, valid_iterator, test_iterator)
    res["num_param"] = count_parameters(model)
    with open("SingleCNN_static_fs%s_%s.json"%(FILTER_SIZE, data_name), "w") as outfile:
        outfile.write(json.dumps(res, indent=4))

    MODE = "non-static"
    model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
    ### pre-train embeddings
    pretrained_embeddings = TEXT.vocab.vectors
    model.embedding.weight.data.copy_(pretrained_embeddings)
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

    res = multi_models(model, train_iterator, valid_iterator, test_iterator)
    res["num_param"] = count_parameters(model)
    with open("SingleCNN_nonstatic_fs%s_%s.json"%(FILTER_SIZE, data_name), "w") as outfile:
        outfile.write(json.dumps(res, indent=4))

    MODE = "spe"
    model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
    ### pre-train embeddings
    pretrained_embeddings = TEXT.vocab.vectors
    model.embedding.weight.data.copy_(pretrained_embeddings)
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

    res = multi_models(model, train_iterator, valid_iterator, test_iterator)
    res["num_param"] = count_parameters(model)
    with open("SingleCNN_spe_fs%s_%s.json"%(FILTER_SIZE, data_name), "w") as outfile:
        outfile.write(json.dumps(res, indent=4))

Begin training model 0
Begin epoch 0
	Train Loss: 1.173 | Train Acc: 57.68%
	 Val. Loss: 0.696 |  Val. Acc: 79.05%
Begin epoch 1
	Train Loss: 0.654 | Train Acc: 77.35%
	 Val. Loss: 0.497 |  Val. Acc: 83.90%
Begin epoch 2
	Train Loss: 0.477 | Train Acc: 84.37%
	 Val. Loss: 0.424 |  Val. Acc: 85.80%
Begin epoch 3
	Train Loss: 0.378 | Train Acc: 87.47%
	 Val. Loss: 0.394 |  Val. Acc: 85.81%
Begin epoch 4
	Train Loss: 0.306 | Train Acc: 90.08%
	 Val. Loss: 0.372 |  Val. Acc: 86.66%
Begin epoch 5
	Train Loss: 0.248 | Train Acc: 92.27%
	 Val. Loss: 0.347 |  Val. Acc: 87.37%
Begin epoch 6
	Train Loss: 0.211 | Train Acc: 93.30%
	 Val. Loss: 0.349 |  Val. Acc: 87.36%
Begin epoch 7
	Train Loss: 0.179 | Train Acc: 94.72%
	 Val. Loss: 0.355 |  Val. Acc: 88.23%
Begin epoch 8
	Train Loss: 0.154 | Train Acc: 95.50%
	 Val. Loss: 0.333 |  Val. Acc: 87.53%
Begin epoch 9
	Train Loss: 0.136 | Train Acc: 96.21%
	 Val. Loss: 0.330 |  Val. Acc: 87.54%
Begin epoch 10
	Train Loss: 0.115 | Train Acc: 96.72%
	 V

In [8]:
# data_name = "TREC"
FILTER_SIZE = 8
for data_name in ["TREC", "SST", "IMDB"]:
    train_iterator, valid_iterator, test_iterator, TEXT, LABEL = load_dataset(data_name, device)
    ### Statis Model without SPE
    INPUT_DIM = len(TEXT.vocab)
    EMBEDDING_DIM = 300
    N_FILTERS = 128
    OUTPUT_DIM = len(LABEL.vocab)
    DROPOUT = 0.5
    PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

    MODE = "static"
    model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
    ### pre-train embeddings
    pretrained_embeddings = TEXT.vocab.vectors
    model.embedding.weight.data.copy_(pretrained_embeddings)
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

    res = multi_models(model, train_iterator, valid_iterator, test_iterator)
    res["num_param"] = count_parameters(model)
    with open("SingleCNN_static_fs%s_%s.json"%(FILTER_SIZE, data_name), "w") as outfile:
        outfile.write(json.dumps(res, indent=4))

    MODE = "non-static"
    model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
    ### pre-train embeddings
    pretrained_embeddings = TEXT.vocab.vectors
    model.embedding.weight.data.copy_(pretrained_embeddings)
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

    res = multi_models(model, train_iterator, valid_iterator, test_iterator)
    res["num_param"] = count_parameters(model)
    with open("SingleCNN_nonstatic_fs%s_%s.json"%(FILTER_SIZE, data_name), "w") as outfile:
        outfile.write(json.dumps(res, indent=4))

    MODE = "spe"
    model = SingleCNN(MODE, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)
    ### pre-train embeddings
    pretrained_embeddings = TEXT.vocab.vectors
    model.embedding.weight.data.copy_(pretrained_embeddings)
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

    res = multi_models(model, train_iterator, valid_iterator, test_iterator)
    res["num_param"] = count_parameters(model)
    with open("SingleCNN_spe_fs%s_%s.json"%(FILTER_SIZE, data_name), "w") as outfile:
        outfile.write(json.dumps(res, indent=4))

Begin training model 0
Begin epoch 0
	Train Loss: 1.091 | Train Acc: 59.47%
	 Val. Loss: 0.624 |  Val. Acc: 79.74%
Begin epoch 1
	Train Loss: 0.564 | Train Acc: 80.36%
	 Val. Loss: 0.455 |  Val. Acc: 84.75%
Begin epoch 2
	Train Loss: 0.395 | Train Acc: 87.22%
	 Val. Loss: 0.389 |  Val. Acc: 87.01%
Begin epoch 3
	Train Loss: 0.290 | Train Acc: 90.76%
	 Val. Loss: 0.363 |  Val. Acc: 87.01%
Begin epoch 4
	Train Loss: 0.217 | Train Acc: 93.77%
	 Val. Loss: 0.345 |  Val. Acc: 86.85%
Begin epoch 5
	Train Loss: 0.160 | Train Acc: 95.69%
	 Val. Loss: 0.334 |  Val. Acc: 87.54%
Begin epoch 6
	Train Loss: 0.134 | Train Acc: 96.56%
	 Val. Loss: 0.354 |  Val. Acc: 86.86%
Begin epoch 7
	Train Loss: 0.107 | Train Acc: 97.38%
	 Val. Loss: 0.337 |  Val. Acc: 87.73%
Begin epoch 8
	Train Loss: 0.085 | Train Acc: 98.09%
	 Val. Loss: 0.336 |  Val. Acc: 87.73%
Begin epoch 9
	Train Loss: 0.074 | Train Acc: 98.48%
	 Val. Loss: 0.338 |  Val. Acc: 86.86%
Begin epoch 10
	Train Loss: 0.061 | Train Acc: 98.76%
	 V